In [1]:
import pandas as pd
import numpy as np

In [3]:
def load_data():
    train_df = pd.read_csv("datasets/train.csv")
    test_df = pd.read_csv("datasets/test.csv")
    return train_df, test_df

train_df, test_df = load_data()

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
is_train         int64
dtype: object

In [29]:
list(train_df.dtypes.values)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('int64')]

In [28]:
str(list(train_df.dtypes.values)[-1])

'int64'

In [21]:
train_df.dtypes.values

array([dtype('int64'), dtype('int64'), dtype('int64'), dtype('O'),
       dtype('O'), dtype('float64'), dtype('int64'), dtype('int64'),
       dtype('O'), dtype('float64'), dtype('O'), dtype('O')], dtype=object)

In [18]:
 list(train_df.dtypes.values())

TypeError: 'numpy.ndarray' object is not callable

In [17]:
for dtype in list(train_df.dtypes.values()):
    print( dtype)
#     if  dtype =="dtype('O')":
        

TypeError: 'numpy.ndarray' object is not callable

In [46]:
train_df["Cabin"].fillna("aaa")

0       aaa
1       C85
2       aaa
3      C123
4       aaa
       ... 
886     aaa
887     B42
888     aaa
889    C148
890     aaa
Name: Cabin, Length: 891, dtype: object

In [51]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
is_train         int64
dtype: object

In [50]:
type(train_df["Cabin"].value_counts()[0])

numpy.int64

In [149]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd


def load_data():
    train_df = pd.read_csv("datasets/train.csv")
    test_df = pd.read_csv("datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(param, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = accuracy_score(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    for tr_inds, val_inds in kf.split(X_train):
        X_tr, y_tr = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y_train.iloc[val_inds]
        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(params, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(Y_test)
        test_preds.append(preds)
    return list(np.sum(np.array(test_preds),axis=0))

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[y_train.columns[0]] = test_preds
    submission.to_csv("submission.csv", index=False)
    # return submission

def main():
    train_df, test_df = load_data()
    id_name = "PassengerId"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2)
    test_preds = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()

if __name__ == '__main__':
    main()


[I 2019-12-11 15:29:56,289] Finished trial#0 resulted in value: 0.8024417801770134. Current best value is 0.8024417801770134 with parameters: {'lambda_l1': 0.01007534045122637, 'lambda_l2': 2.8537356571466826e-07, 'num_leaves': 180, 'feature_fraction': 0.7807180524166191, 'bagging_fraction': 0.4225197119630312, 'bagging_freq': 1, 'min_child_samples': 69}.
[I 2019-12-11 15:29:57,580] Finished trial#1 resulted in value: 0.8215240725629276. Current best value is 0.8215240725629276 with parameters: {'lambda_l1': 0.1871875138806655, 'lambda_l2': 7.694254834511715e-05, 'num_leaves': 239, 'feature_fraction': 0.9516662769431276, 'bagging_fraction': 0.949755419696052, 'bagging_freq': 5, 'min_child_samples': 38}.


val_score→0.8215240725629276


In [150]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd
import numpy as np


def load_data():
    train_df = pd.read_csv("datasets/train.csv")
    test_df = pd.read_csv("datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(param, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = accuracy_score(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    for tr_inds, val_inds in kfold.split(X):
        X_tr, y_tr = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(params, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(Y_test)
        test_preds.append(preds)
    return list(np.sum(np.array(test_preds),axis=0))

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[y_train.columns[0]] = test_preds
    submission.to_csv("submission.csv", index=False)

if __name__=='__main__':
    train_df, test_df = load_data()
    id_name = "PassengerId"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2)
    test_preds = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()


[I 2019-12-11 16:58:47,186] Finished trial#0 resulted in value: 0.8215115184232. Current best value is 0.8215115184232 with parameters: {'lambda_l1': 0.0410629213560687, 'lambda_l2': 0.0420827881795705, 'num_leaves': 27, 'feature_fraction': 0.48507022763086965, 'bagging_fraction': 0.806331767716109, 'bagging_freq': 2, 'min_child_samples': 19}.
[I 2019-12-11 16:58:49,444] Finished trial#1 resulted in value: 0.8282656455966355. Current best value is 0.8282656455966355 with parameters: {'lambda_l1': 0.009996579283141427, 'lambda_l2': 2.1546613695485014e-05, 'num_leaves': 59, 'feature_fraction': 0.5974896029112862, 'bagging_fraction': 0.9033984180208278, 'bagging_freq': 2, 'min_child_samples': 14}.


val_score→0.8282656455966355


In [151]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd
import numpy as np


def load_data():
    train_df = pd.read_csv("datasets/train.csv")
    test_df = pd.read_csv("datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(param, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = accuracy_score(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    for tr_inds, val_inds in kfold.split(X):
        X_tr, y_tr = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(params, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(Y_test)
        test_preds.append(preds)
    return list(np.sum(np.array(test_preds),axis=0))

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[Y.columns[0]] = test_preds
    submission.to_csv("submission.csv", index=False)

if __name__=='__main__':
    train_df, test_df = load_data()
    id_name = "PassengerId"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2)
    test_preds = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()


[I 2019-12-11 17:03:03,396] Finished trial#0 resulted in value: 0.8248634737304626. Current best value is 0.8248634737304626 with parameters: {'lambda_l1': 8.391185371922389e-05, 'lambda_l2': 0.22957964356011817, 'num_leaves': 100, 'feature_fraction': 0.9050349829051887, 'bagging_fraction': 0.950857016647558, 'bagging_freq': 6, 'min_child_samples': 59}.
[I 2019-12-11 17:03:04,451] Finished trial#1 resulted in value: 0.8170045822610006. Current best value is 0.8248634737304626 with parameters: {'lambda_l1': 8.391185371922389e-05, 'lambda_l2': 0.22957964356011817, 'num_leaves': 100, 'feature_fraction': 0.9050349829051887, 'bagging_fraction': 0.950857016647558, 'bagging_freq': 6, 'min_child_samples': 59}.


val_score→0.8248634737304626


In [156]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd
import numpy as np


def load_data():
    train_df = pd.read_csv("house_datasets/train.csv")
    test_df = pd.read_csv("house_datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    print(param)

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(param, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = mean_squared_error(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    for tr_inds, val_inds in kfold.split(X):
        X_tr, y_tr = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train)
        model = lgb.train(params, lgb_train, valid_sets=lgb_eval, verbose_eval=False)
        preds = model.predict(Y_test)
        test_preds.append(preds)
    return list(np.sum(np.array(test_preds),axis=0))

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[Y.columns[0]] = test_preds
    submission.to_csv("classify_submission.csv", index=False)

if __name__=='__main__':
    train_df, test_df = load_data()
    id_name = "Id"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study()
    study.optimize(objective, n_trials=30)
    test_preds = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()


{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.3732283768904134, 'lambda_l2': 0.0398278591638978, 'num_leaves': 59, 'feature_fraction': 0.5636595595976339, 'bagging_fraction': 0.6323649528870036, 'bagging_freq': 3, 'min_child_samples': 48}


[I 2019-12-11 17:17:04,687] Finished trial#0 resulted in value: 39038905866.37397. Current best value is 39038905866.37397 with parameters: {'lambda_l1': 0.3732283768904134, 'lambda_l2': 0.0398278591638978, 'num_leaves': 59, 'feature_fraction': 0.5636595595976339, 'bagging_fraction': 0.6323649528870036, 'bagging_freq': 3, 'min_child_samples': 48}.


{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.00976786167451293, 'lambda_l2': 0.1290934220799596, 'num_leaves': 57, 'feature_fraction': 0.7613732027534676, 'bagging_fraction': 0.5744022343599056, 'bagging_freq': 6, 'min_child_samples': 42}


[I 2019-12-11 17:17:05,296] Finished trial#1 resulted in value: 39038905866.37397. Current best value is 39038905866.37397 with parameters: {'lambda_l1': 0.3732283768904134, 'lambda_l2': 0.0398278591638978, 'num_leaves': 59, 'feature_fraction': 0.5636595595976339, 'bagging_fraction': 0.6323649528870036, 'bagging_freq': 3, 'min_child_samples': 48}.


{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 9.829111058576442e-07, 'lambda_l2': 4.606908184658166, 'num_leaves': 176, 'feature_fraction': 0.5370511802502533, 'bagging_fraction': 0.7991571336097336, 'bagging_freq': 7, 'min_child_samples': 15}


...

{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.0032861829925461767, 'lambda_l2': 2.0559701671882483e-05, 'num_leaves': 52, 'feature_fraction': 0.4528970370496347, 'bagging_fraction': 0.5723099935747089, 'bagging_freq': 6, 'min_child_samples': 20}


[I 2019-12-11 17:17:22,756] Finished trial#28 resulted in value: 39038905866.37397. Current best value is 39038905866.37397 with parameters: {'lambda_l1': 0.3732283768904134, 'lambda_l2': 0.0398278591638978, 'num_leaves': 59, 'feature_fraction': 0.5636595595976339, 'bagging_fraction': 0.6323649528870036, 'bagging_freq': 3, 'min_child_samples': 48}.


{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 1.0054806342804425, 'lambda_l2': 0.0011545002041276258, 'num_leaves': 53, 'feature_fraction': 0.7881303348101888, 'bagging_fraction': 0.9684376309214346, 'bagging_freq': 3, 'min_child_samples': 5}


[I 2019-12-11 17:17:23,413] Finished trial#29 resulted in value: 39038905866.37397. Current best value is 39038905866.37397 with parameters: {'lambda_l1': 0.3732283768904134, 'lambda_l2': 0.0398278591638978, 'num_leaves': 59, 'feature_fraction': 0.5636595595976339, 'bagging_fraction': 0.6323649528870036, 'bagging_freq': 3, 'min_child_samples': 48}.


val_score→39038905866.37397


In [113]:
study.best_params

{'lambda_l1': 0.0006366732455576157,
 'lambda_l2': 0.5434592970214148,
 'num_leaves': 188,
 'feature_fraction': 0.7052767511832432,
 'bagging_fraction': 0.8140017738784788,
 'bagging_freq': 6,
 'min_child_samples': 34}

0.8204506936162199

In [100]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
for tr_inds, val_inds in kf.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        break

In [76]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,155,1,22.0,1,0,720,7.2500,186,2
1,2,1,286,0,38.0,1,0,816,71.2833,106,0
2,3,3,523,0,26.0,0,0,914,7.9250,186,2
3,4,1,422,0,35.0,1,0,65,53.1000,70,2
4,5,3,22,1,35.0,0,0,649,8.0500,186,2


In [77]:
Y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [ ]:
y_column = []

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score


def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    print(structure_params)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(X)):
        tr_x, tr_y = X[tr_inds],Y[tr_inds]
        vl_x, vl_y = X[val_inds],Y[val_inds]
        gbm = lgb.train(param, dtrain)
        preds = gbm.predict(test_x)
        pred_labels = np.rint(preds)
        score = accuracy_score(test_y, pred_labels)
        scores.append(score)
    return mean(scores)

In [170]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd
import numpy as np


def load_data():
    train_df = pd.read_csv("datasets/train.csv")
    test_df = pd.read_csv("datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train,y_train,early_stopping_rounds=10,eval_set=[[X_eval, y_eval]],verbose=0)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = accuracy_score(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    feature_importance_df = pd.DataFrame()
    feats = X.columns
    n_fold = 1
    for tr_inds, val_inds in kfold.split(X):
        X_tr, y_tr = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train,y_train,early_stopping_rounds=10,eval_set=[[X_eval, y_eval]])
        preds = model.predict(Y_test)
        test_preds.append(preds)
    feature_importance_df["feature"] = feats
    feature_importance_df["importance"] = model.feature_importances_
    return list(np.sum(np.array(test_preds),axis=0)),feature_importance_df

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[Y.columns[0]] = test_preds
    submission.to_csv("classify_submission.csv", index=False)

if __name__=='__main__':
    train_df, test_df = load_data()
    id_name = "PassengerId"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study()
    study.optimize(objective, n_trials=2)
    test_preds,feature_importance_df = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()


[I 2019-12-11 20:35:00,244] Finished trial#0 resulted in value: 0.8035842068922227. Current best value is 0.8035842068922227 with parameters: {'lambda_l1': 0.2672638076707509, 'lambda_l2': 0.0015885266596271889, 'num_leaves': 186, 'feature_fraction': 0.40844140963336517, 'bagging_fraction': 0.7395204488916367, 'bagging_freq': 2, 'min_child_samples': 86}.
[I 2019-12-11 20:35:00,913] Finished trial#1 resulted in value: 0.7923419747661792. Current best value is 0.7923419747661792 with parameters: {'lambda_l1': 0.14555331012540598, 'lambda_l2': 1.4099140599129005e-07, 'num_leaves': 239, 'feature_fraction': 0.4087073243050536, 'bagging_fraction': 0.7336875423402733, 'bagging_freq': 6, 'min_child_samples': 100}.


[  1   5   8  14  27  30  31  34  37  40  55  60  62  75  77  79  85  97
 101 103 113 141 142 144 145 150 158 175 181 193 196 200 202 214 215 222
 230 231 236 247 249 251 252 255 262 264 266 267 270 278 279 285 293 294
 298 299 301 310 311 312 316 317 318 319 342 346 350 352 356 362 367 372
 380 384 389 390 395 397 399 412 413 418 425 432 440 458 474 477 483 484
 487 489 494 495 496 500 501 503 505 511 513 516 519 522 523 530 531 535
 538 542 545 561 564 566 567 570 587 596 609 612 614 619 627 631 632 634
 635 638 642 643 644 648 654 655 656 658 666 676 681 683 686 693 694 700
 704 708 712 715 725 726 728 740 750 760 762 764 766 768 771 778 780 784
 796 803 808 815 818 819 822 833 837 838 839 848 877 880 883 887 890]
[1]	valid_0's binary_logloss: 0.652849
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.637663
[3]	valid_0's binary_logloss: 0.62469
[4]	valid_0's binary_logloss: 0.61603
[5]	valid_0's binary_logloss: 0.609481
[6]	valid_0's bina

[65]	valid_0's binary_logloss: 0.447433
[66]	valid_0's binary_logloss: 0.447345
[67]	valid_0's binary_logloss: 0.445639
[68]	valid_0's binary_logloss: 0.445024
[69]	valid_0's binary_logloss: 0.443968
[70]	valid_0's binary_logloss: 0.443872
[71]	valid_0's binary_logloss: 0.442917
[72]	valid_0's binary_logloss: 0.442385
[73]	valid_0's binary_logloss: 0.440524
[74]	valid_0's binary_logloss: 0.437154
[75]	valid_0's binary_logloss: 0.436513
[76]	valid_0's binary_logloss: 0.435214
[77]	valid_0's binary_logloss: 0.434062
[78]	valid_0's binary_logloss: 0.433848
[79]	valid_0's binary_logloss: 0.433012
[80]	valid_0's binary_logloss: 0.43202
[81]	valid_0's binary_logloss: 0.431016
[82]	valid_0's binary_logloss: 0.430465
[83]	valid_0's binary_logloss: 0.429123
[84]	valid_0's binary_logloss: 0.429218
[85]	valid_0's binary_logloss: 0.429247
[86]	valid_0's binary_logloss: 0.4279
[87]	valid_0's binary_logloss: 0.428436
[88]	valid_0's binary_logloss: 0.428912
[89]	valid_0's binary_logloss: 0.428587
[90

[27]	valid_0's binary_logloss: 0.482238
[28]	valid_0's binary_logloss: 0.480488
[29]	valid_0's binary_logloss: 0.479519
[30]	valid_0's binary_logloss: 0.475769
[31]	valid_0's binary_logloss: 0.471597
[32]	valid_0's binary_logloss: 0.470261
[33]	valid_0's binary_logloss: 0.468791
[34]	valid_0's binary_logloss: 0.467409
[35]	valid_0's binary_logloss: 0.465498
[36]	valid_0's binary_logloss: 0.464996
[37]	valid_0's binary_logloss: 0.46453
[38]	valid_0's binary_logloss: 0.463269
[39]	valid_0's binary_logloss: 0.46249
[40]	valid_0's binary_logloss: 0.461152
[41]	valid_0's binary_logloss: 0.459734
[42]	valid_0's binary_logloss: 0.458255
[43]	valid_0's binary_logloss: 0.458204
[44]	valid_0's binary_logloss: 0.458173
[45]	valid_0's binary_logloss: 0.457139
[46]	valid_0's binary_logloss: 0.457025
[47]	valid_0's binary_logloss: 0.456664
[48]	valid_0's binary_logloss: 0.456858
[49]	valid_0's binary_logloss: 0.456043
[50]	valid_0's binary_logloss: 0.455707
[51]	valid_0's binary_logloss: 0.45615
[52

In [173]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
from statistics import mean
import pandas as pd
import numpy as np


def load_data():
    train_df = pd.read_csv("house_datasets/train.csv")
    test_df = pd.read_csv("house_datasets/test.csv")
    return train_df, test_df


def preprocessing(train_df,test_df, id_name):
    train_df["is_train"] = 1
    test_df["is_train"] = 0
    target = set(list(train_df.columns)) ^ set(list(test_df.columns))
    target_df = train_df[target]
    train_df = train_df.drop(target,axis=1)
    all_data = pd.concat([train_df,test_df])
    # objectカラムの取得
    change_columns =[]
    for index_name, dtype in zip(list(train_df.dtypes.index),list(train_df.dtypes.values)):
        if  str(dtype) =='object':
            change_columns.append(index_name)
    train_df[change_columns] = list(train_df[change_columns].fillna("欠損"))
    target_column = train_df[change_columns]
    for column in change_columns:
        le = LabelEncoder()
        select_df = all_data[column].astype(str)
        le.fit(select_df)
        all_data[column] = le.transform(select_df)
    all_data = all_data.drop(id_name,axis=1)
    train_df = all_data[all_data["is_train"]==1].drop("is_train",axis=1)
    test_df = all_data[all_data["is_train"]==0].drop("is_train",axis=1)
    return train_df, test_df, target_df

def objective(trial):
    # optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kfold = KFold(5, random_state = 0, shuffle = True)
    scores = []
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train,y_train,early_stopping_rounds=10,eval_set=[[X_eval, y_eval]],verbose=0)
        preds = model.predict(X_eval)
        pred_labels = np.rint(preds)
        score = accuracy_score(y_eval, pred_labels)
        scores.append(score)
    return mean(scores)

def make_prediction(params):
    kfold = KFold(5, random_state = 0, shuffle = True)
    test_preds = []
    feature_importance_df = pd.DataFrame()
    feats = X.columns
    n_fold = 1
    for tr_inds, val_inds in kfold.split(X):
        X_train, y_train = X.iloc[tr_inds],Y.iloc[tr_inds]
        X_eval, y_eval = X.iloc[val_inds],Y.iloc[val_inds]
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train,y_train,early_stopping_rounds=10,eval_set=[[X_eval, y_eval]])
        preds = model.predict(Y_test)
        test_preds.append(preds)
    feature_importance_df["feature"] = feats
    feature_importance_df["importance"] = model.feature_importances_
    return list(np.sum(np.array(test_preds),axis=0)),feature_importance_df

def to_csv():
    submission = pd.DataFrame()
    submission[id_name] = test_df[id_name]
    submission[Y.columns[0]] = test_preds
    submission.to_csv("classify_submission.csv", index=False)

if __name__=='__main__':
    train_df, test_df = load_data()
    id_name = "Id"
    X, Y_test, Y = preprocessing(train_df, test_df, id_name)

    study = optuna.create_study()
    study.optimize(objective, n_trials=2)
    test_preds,feature_importance_df = make_prediction(study.best_params)
    print("val_score→"+str(study.best_value))
    to_csv()


[I 2019-12-11 20:40:12,670] Finished trial#0 resulted in value: 0.0. Current best value is 0.0 with parameters: {'lambda_l1': 8.984782016185405e-05, 'lambda_l2': 5.737920679394895e-06, 'num_leaves': 73, 'feature_fraction': 0.6395302472894522, 'bagging_fraction': 0.6081379325901611, 'bagging_freq': 3, 'min_child_samples': 84}.
[I 2019-12-11 20:40:14,834] Finished trial#1 resulted in value: 0.0. Current best value is 0.0 with parameters: {'lambda_l1': 8.984782016185405e-05, 'lambda_l2': 5.737920679394895e-06, 'num_leaves': 73, 'feature_fraction': 0.6395302472894522, 'bagging_fraction': 0.6081379325901611, 'bagging_freq': 3, 'min_child_samples': 84}.


ValueError: y contains previously unseen labels: [68500, 79500, 82500, 84900, 87500, 89471, 90350, 92900, 99900, 101000, 104000, 105900, 106250, 107900, 108959, 116050, 119200, 123600, 128200, 134450, 136000, 136905, 137900, 139500, 142953, 145500, 145900, 148800, 149350, 150500, 155900, 156500, 158500, 163900, 164990, 165600, 175900, 178900, 181500, 181900, 182900, 183900, 185900, 186000, 187750, 188500, 193500, 193879, 194201, 198500, 200500, 200624, 202665, 206300, 213250, 213490, 216837, 224500, 229456, 230500, 233000, 239900, 243000, 245350, 248900, 249700, 251000, 263000, 265000, 277500, 282922, 301000, 306000, 313000, 316600, 319900, 324000, 367294, 370878, 372500, 385000, 386250, 392000, 403000, 412500, 415298, 438780, 538000, 555000, 745000]

In [172]:
feature_importance_df.sort_values('importance', ascending=False)

,feature,importance
1,Name,77
6,Ticket,76
7,Fare,76
3,Age,63
2,Sex,30
8,Cabin,21
0,Pclass,18
9,Embarked,10
4,SibSp,8
5,Parch,5


In [167]:
##今全部一緒だがこれで大丈夫かは要確認する
feature_importance_df

,feature,importance,fold,feature,importance,fold,feature,importance,fold,feature,importance,fold,feature,importance,fold
0,Pclass,37,1,Pclass,37,2,Pclass,37,3,Pclass,37,4,Pclass,37,5
1,Name,193,1,Name,193,2,Name,193,3,Name,193,4,Name,193,5
2,Sex,47,1,Sex,47,2,Sex,47,3,Sex,47,4,Sex,47,5
3,Age,231,1,Age,231,2,Age,231,3,Age,231,4,Age,231,5
4,SibSp,21,1,SibSp,21,2,SibSp,21,3,SibSp,21,4,SibSp,21,5
5,Parch,11,1,Parch,11,2,Parch,11,3,Parch,11,4,Parch,11,5
6,Ticket,221,1,Ticket,221,2,Ticket,221,3,Ticket,221,4,Ticket,221,5
7,Fare,215,1,Fare,215,2,Fare,215,3,Fare,215,4,Fare,215,5
8,Cabin,50,1,Cabin,50,2,Cabin,50,3,Cabin,50,4,Cabin,50,5
9,Embarked,22,1,Embarked,22,2,Embarked,22,3,Embarked,22,4,Embarked,22,5


In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = feats
fold_importance_df["importance"] = clf.feature_importances_
fold_importance_df["fold"] = n_fold + 1
feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

In [154]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_train
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [ ]:
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=47)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=47)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            #is_unbalance=True,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=32,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.04,
            reg_lambda=0.073,
            min_split_gain=0.0222415,
            min_child_weight=40,
            silent=-1,
            verbose=-1,
            #scale_pos_weight=11
            )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 1000, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df